## ChatMessageHistory

In [ ]:
import { ChatMessageHistory } from "langchain/stores/message/in_memory";
import { HumanMessage, AIMessage } from "@langchain/core/messages";

const history = new ChatMessageHistory();
await history.addMessage(new HumanMessage("hi"));
await history.addMessage(new AIMessage("What can I do for you?"));

const messages = await history.getMessages();

console.log(messages);


## 手动维护 chat history

In [ ]:
import { load } from "dotenv";
const env = await load();

const process = {
    env
}

In [ ]:
import { ChatPromptTemplate, MessagesPlaceholder } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";

const chatModel = new ChatOpenAI();
const prompt = ChatPromptTemplate.fromMessages([
    ["system", `You are a helpful assistant. Answer all questions to the best of your ability.
    You are talkative and provides lots of specific details from its context. 
    If the you does not know the answer to a question, it truthfully says you do not know.`],
    new MessagesPlaceholder("history_message"),
]);

const chain = prompt.pipe(chatModel);

In [ ]:
import { ChatMessageHistory } from "langchain/stores/message/in_memory";
import { HumanMessage, AIMessage } from "@langchain/core/messages";

const history = new ChatMessageHistory();
await history.addMessage(new HumanMessage("hi, my name is Kai"));

const res1 = await chain.invoke({
    history_message: await history.getMessages()
})


In [ ]:
res1

In [ ]:
await history.addMessage(res1)
await history.addMessage(new HumanMessage("What is my name?"));

In [ ]:
await history.getMessages()

In [ ]:
const res2 = await chain.invoke({
    history_message: await history.getMessages()
})

In [ ]:
res2

In [ ]:
console.log(res2.content)

## 自动维护 chat history

In [ ]:
import { RunnableWithMessageHistory } from "@langchain/core/runnables";
import { ChatPromptTemplate, MessagesPlaceholder } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import { ChatMessageHistory } from "langchain/stores/message/in_memory";

const chatModel = new ChatOpenAI();
const prompt = ChatPromptTemplate.fromMessages([
    ["system", "You are a helpful assistant. Answer all questions to the best of your ability."],
    new MessagesPlaceholder("history_message"),
    ["human","{input}"]
]);

const history = new ChatMessageHistory();
const chain = prompt.pipe(chatModel)

const chainWithHistory = new RunnableWithMessageHistory({
  runnable: chain,
  getMessageHistory: (_sessionId) => history,
  inputMessagesKey: "input",
  historyMessagesKey: "history_message",
});

In [ ]:
const res1 = await chainWithHistory.invoke({
    input: "hi, my name is Kai"
},{
    configurable: { sessionId: "none" }
})

In [ ]:
res1

In [ ]:
const res2 = await chainWithHistory.invoke({
    input: "我的名字叫什么？"
},{
    configurable: { sessionId: "none" }
})

In [ ]:
res2

In [ ]:
await history.getMessages()

## 自动生成 chat history 摘要

In [ ]:
import { RunnableWithMessageHistory } from "@langchain/core/runnables";
import { ChatPromptTemplate, MessagesPlaceholder } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import { ChatMessageHistory } from "langchain/stores/message/in_memory";
import { RunnableSequence } from "@langchain/core/runnables";
import { RunnablePassthrough } from "@langchain/core/runnables";
import { StringOutputParser } from "@langchain/core/output_parsers";
import { getBufferString } from "@langchain/core/messages";

In [ ]:
const summaryModel = new ChatOpenAI();
const summaryPrompt = ChatPromptTemplate.fromTemplate(`
Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:
`); 

const summaryChain = RunnableSequence.from([
    summaryPrompt,
    summaryModel,
    new StringOutputParser(),
])


In [ ]:
const newSummary = await summaryChain.invoke({
    "",
    new_lines: "I'm 18"
})

In [ ]:
console.log(res)

In [ ]:
await summaryChain.invoke({
    summary: res,
    new_lines: "I'm male"
})

In [ ]:
const chatModel = new ChatOpenAI();
const chatPrompt = ChatPromptTemplate.fromMessages([
    ["system", `You are a helpful assistant. Answer all questions to the best of your ability.

    Here is the chat history summary:
    {history_summary}
    `],
    ["human","{input}"]
]);
let summary = ""
const history = new ChatMessageHistory();

const chatChain = RunnableSequence.from([
    {
        input: new RunnablePassthrough({
             func: (input) => history.addUserMessage(input)
        })
    },
    RunnablePassthrough.assign({
        history_summary: () => summary
    }),
    chatPrompt,
    chatModel,
    new StringOutputParser(),
    new RunnablePassthrough({
        func: async (input) => {
            history.addAIChatMessage(input)
            const messages = await history.getMessages()
            const new_lines = getBufferString(messages)
            const newSummary = await summaryChain.invoke({
                summary,
                new_lines
            })
            console.log(summary, input, messages, new_lines, newSummary)
            history.clear()
            summary = newSummary      
        }
    })
])


In [ ]:
await chatChain.invoke("我现在饿了")

In [ ]:
console.log(summary)

In [ ]:
await chatChain.invoke("我今天想吃方便面")

In [ ]:
console.log(summary)

In [ ]:
import { RunnableMap } from "@langchain/core/runnables"

const mapChain = RunnableMap.from({
    a: () => "a",
    b: () => "b"
})

const res = await mapChain.invoke()

In [ ]:
res